In [1]:
import pandas as pd
train = pd.read_csv('./ner_work/train_data_public.csv')
test = pd.read_csv('./ner_work/test_public.csv')
train

,id,text,BIO_anno,class
0,0,交行14年用过，半年准备提额，却直接被降到1Ｋ，半年期间只T过一次三千，其它全部真实消费，第...,B-BANK I-BANK O O O O O O O O O O B-COMMENTS_N...,0
1,1,单标我有了，最近visa双标返现活动好,B-PRODUCT I-PRODUCT O O O O O O B-PRODUCT I-PR...,1
2,2,建设银行提额很慢的……,B-BANK I-BANK I-BANK I-BANK B-COMMENTS_N I-COM...,0
3,3,我的怎么显示0.25费率，而且不管分多少期都一样费率，可惜只有69k,O O O O O O O O O O B-COMMENTS_N I-COMMENTS_N ...,2
4,4,利率不错，可以撸,B-COMMENTS_N I-COMMENTS_N B-COMMENTS_ADJ I-COM...,1
...,...,...,...,...
7523,7523,我鼎级拒了,O O O B-COMMENTS_ADJ O,2
7524,7524,一打一个准，准胜，看激活信用卡时那协议，全是对银行有利的,O O O O O O O O O O B-COMMENTS_N I-COMMENTS_N ...,2
7525,7525,招行分期白80k,B-BANK I-BANK B-PRODUCT I-PRODUCT I-PRODUCT O O O,2
7526,7526,5万，额度还行吧没毕业哦,O O O B-COMMENTS_N I-COMMENTS_N O O O O O O O,2


In [2]:
!pip install paddlehub==2.1.1

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.4/211.4 KB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 7.3 MB/s eta 0:00:0000:0100:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 3.7 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: paddlehub
    Found existing installation: paddlehub 2.0.4
    Uninstalling paddlehub-2.0.4:
      Successfully uninstalled paddlehub-2.0.4


In [3]:
import paddle
import paddlehub as hub
from paddlehub.datasets.base_nlp_dataset import BaseNLPDataset
from typing import Dict, List, Optional, Union, Tuple
from paddlehub.datasets.base_nlp_dataset import TextClassificationDataset
from paddlehub.text.bert_tokenizer import BertTokenizer
from paddlehub.text.tokenizer import CustomTokenizer

# 自定义数据集
class MyDataset(TextClassificationDataset):
    """DemoDataset"""
    def __init__(self, tokenizer: Union[BertTokenizer, CustomTokenizer], max_seq_len: int = 128, mode: str = 'train'):
        # 数据集存放位置
        base_path = "/home/aistudio/ner_work"
        if mode == 'train':
            data_file = 'ernie_train.tsv'
        elif mode == 'test':
            data_file = 'ernie_test.tsv'
        else:
            data_file = 'ernie_dev.tsv'
        super().__init__(
            base_path=base_path,
            tokenizer=tokenizer,
            max_seq_len=max_seq_len,
            mode=mode,
            data_file=data_file,
            label_list=["0", "1", "2"],
            is_file_with_header=True)

#Step2，使用PaddleHub
# ernie1.0, 2.0, 3.0
# PaddleHub 预训练包，直接用 + FineTunning, 多种预训练模型 sequence classification => sentence classification
model = hub.Module(name='ernie_tiny', version='2.0.1', task='seq-cls', num_classes=3)
#数据集设置
train_dataset = MyDataset(tokenizer=model.get_tokenizer(), max_seq_len=512, mode='train')
#test_dataset = MyDataset(tokenizer=model.get_tokenizer(), max_seq_len=512, mode='test')
#val_dataset
#Step3，设置优化器，运行配置
optimizer = paddle.optimizer.AdamW(learning_rate=5e-5, parameters=model.parameters())
trainer = hub.Trainer(model, optimizer, checkpoint_dir='./ernie_checkpoint', use_gpu=True)


/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/__init__.py:107: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import MutableMapping
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/rcsetup.py:20: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Iterable, Mapping
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/colors.py:53: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Sized


Download https://bj.bcebos.com/paddlehub/paddlehub_dev/ernie_tiny_2.0.1.tar.gz
[##################################################] 100.00%
Decompress /home/aistudio/.paddlehub/tmp/tmpjz4g2i0v/ernie_tiny_2.0.1.tar.gz
[##################################################] 100.00%


[2022-04-29 21:31:29,036] [    INFO] - Successfully installed ernie_tiny-2.0.1
[2022-04-29 21:31:29,040] [    INFO] - Downloading https://paddlenlp.bj.bcebos.com/models/transformers/ernie_tiny/ernie_tiny.pdparams and saved to /home/aistudio/.paddlenlp/models/ernie-tiny
[2022-04-29 21:31:29,042] [    INFO] - Downloading ernie_tiny.pdparams from https://paddlenlp.bj.bcebos.com/models/transformers/ernie_tiny/ernie_tiny.pdparams
100%|██████████| 354158/354158 [00:12<00:00, 28607.18it/s]
W0429 21:31:41.524824   243 device_context.cc:447] Please NOTE: device: 0, GPU Compute Capability: 7.0, Driver API Version: 10.1, Runtime API Version: 10.1
W0429 21:31:41.529733   243 device_context.cc:465] device: 0, cuDNN Version: 7.6.
[2022-04-29 21:31:49,399] [    INFO] - Downloading vocab.txt from https://paddlenlp.bj.bcebos.com/models/transformers/ernie_tiny/vocab.txt
100%|██████████| 459/459 [00:00<00:00, 3700.25it/s]
[2022-04-29 21:31:49,642] [    INFO] - Downloading spm_cased_simp_sampled.model fro

In [ ]:
help(trainer)

In [12]:
# 1,自定义数据集 2，sampling dataloader 3, 参数配置 4, 训练过程
trainer.train(
    train_dataset, # 只针对train
    epochs=3,
    batch_size=32,
    save_interval=1,
)

[2022-04-15 22:26:15,419] [   TRAIN] - Epoch=1/3, Step=10/236 loss=0.6749 acc=0.7406 lr=0.000050 step/sec=2.48 | ETA 00:04:45
[2022-04-15 22:26:19,050] [   TRAIN] - Epoch=1/3, Step=20/236 loss=0.4773 acc=0.8781 lr=0.000050 step/sec=2.75 | ETA 00:04:31
[2022-04-15 22:26:22,675] [   TRAIN] - Epoch=1/3, Step=30/236 loss=0.4221 acc=0.8625 lr=0.000050 step/sec=2.76 | ETA 00:04:26
[2022-04-15 22:26:26,315] [   TRAIN] - Epoch=1/3, Step=40/236 loss=0.4665 acc=0.8250 lr=0.000050 step/sec=2.75 | ETA 00:04:24
[2022-04-15 22:26:29,962] [   TRAIN] - Epoch=1/3, Step=50/236 loss=0.3869 acc=0.8562 lr=0.000050 step/sec=2.74 | ETA 00:04:23
[2022-04-15 22:26:33,610] [   TRAIN] - Epoch=1/3, Step=60/236 loss=0.3604 acc=0.8719 lr=0.000050 step/sec=2.74 | ETA 00:04:22
[2022-04-15 22:26:37,256] [   TRAIN] - Epoch=1/3, Step=70/236 loss=0.3046 acc=0.8844 lr=0.000050 step/sec=2.74 | ETA 00:04:21
[2022-04-15 22:26:40,910] [   TRAIN] - Epoch=1/3, Step=80/236 loss=0.3618 acc=0.8594 lr=0.000050 step/sec=2.74 | ETA 0

In [4]:
label_map = {0: 0, 1: 1, 2: 2}
model = hub.Module(
    name='ernie_tiny',
    version='2.0.1',
    task='seq-cls',
    load_checkpoint='./ernie_checkpoint/epoch_3/model.pdparams', 
    label_map=label_map)
model


[2022-04-29 21:32:06,375] [    INFO] - Already cached /home/aistudio/.paddlenlp/models/ernie-tiny/ernie_tiny.pdparams
[2022-04-29 21:32:10,131] [    INFO] - Loaded parameters from /home/aistudio/ernie_checkpoint/epoch_3/model.pdparams


ErnieTiny(
  (model): ErnieForSequenceClassification(
    (ernie): ErnieModel(
      (embeddings): ErnieEmbeddings(
        (word_embeddings): Embedding(50006, 1024, padding_idx=0, sparse=False)
        (position_embeddings): Embedding(600, 1024, sparse=False)
        (token_type_embeddings): Embedding(2, 1024, sparse=False)
        (layer_norm): LayerNorm(normalized_shape=[1024], epsilon=1e-05)
        (dropout): Dropout(p=0.1, axis=None, mode=upscale_in_train)
      )
      (encoder): TransformerEncoder(
        (layers): LayerList(
          (0): TransformerEncoderLayer(
            (self_attn): MultiHeadAttention(
              (q_proj): Linear(in_features=1024, out_features=1024, dtype=float32)
              (k_proj): Linear(in_features=1024, out_features=1024, dtype=float32)
              (v_proj): Linear(in_features=1024, out_features=1024, dtype=float32)
              (out_proj): Linear(in_features=1024, out_features=1024, dtype=float32)
            )
            (linear1): Lin

In [5]:
train['text']

,id,text,BIO_anno,class
0,0,交行14年用过，半年准备提额，却直接被降到1Ｋ，半年期间只T过一次三千，其它全部真实消费，第...,B-BANK I-BANK O O O O O O O O O O B-COMMENTS_N...,0
1,1,单标我有了，最近visa双标返现活动好,B-PRODUCT I-PRODUCT O O O O O O B-PRODUCT I-PR...,1
2,2,建设银行提额很慢的……,B-BANK I-BANK I-BANK I-BANK B-COMMENTS_N I-COM...,0
3,3,我的怎么显示0.25费率，而且不管分多少期都一样费率，可惜只有69k,O O O O O O O O O O B-COMMENTS_N I-COMMENTS_N ...,2
4,4,利率不错，可以撸,B-COMMENTS_N I-COMMENTS_N B-COMMENTS_ADJ I-COM...,1
...,...,...,...,...
7523,7523,我鼎级拒了,O O O B-COMMENTS_ADJ O,2
7524,7524,一打一个准，准胜，看激活信用卡时那协议，全是对银行有利的,O O O O O O O O O O B-COMMENTS_N I-COMMENTS_N ...,2
7525,7525,招行分期白80k,B-BANK I-BANK B-PRODUCT I-PRODUCT I-PRODUCT O O O,2
7526,7526,5万，额度还行吧没毕业哦,O O O B-COMMENTS_N I-COMMENTS_N O O O O O O O,2


In [ ]:
#test = pd.read_csv('./ner_work/test_public.csv')
train_results = model.predict([[x] for x in train['text'].values], max_seq_len=512, batch_size=32, use_gpu=True)

[2022-04-29 21:33:46,031] [    INFO] - Found /home/aistudio/.paddlenlp/models/ernie-tiny/vocab.txt
[2022-04-29 21:33:46,033] [    INFO] - Found /home/aistudio/.paddlenlp/models/ernie-tiny/spm_cased_simp_sampled.model
[2022-04-29 21:33:46,036] [    INFO] - Found /home/aistudio/.paddlenlp/models/ernie-tiny/dict.wordseg.pickle


In [ ]:
train_results

In [6]:
test_result = pd.read_csv('./baselin1.csv')
#test_result['class'] = 

test = pd.read_csv('./ner_work/test_public.csv')
results = model.predict([[x] for x in test['text'].values], max_seq_len=512, batch_size=32, use_gpu=True)

[2022-04-22 19:56:42,329] [    INFO] - Found /home/aistudio/.paddlenlp/models/ernie-tiny/vocab.txt
[2022-04-22 19:56:42,332] [    INFO] - Found /home/aistudio/.paddlenlp/models/ernie-tiny/spm_cased_simp_sampled.model
[2022-04-22 19:56:42,335] [    INFO] - Found /home/aistudio/.paddlenlp/models/ernie-tiny/dict.wordseg.pickle


In [11]:
#results
test_result['class'] = results
test_result

,id,BIO_anno,class
0,0,O O O O B-COMMENTS_N I-COMMENTS_N O O O O O O ...,2
1,1,B-COMMENTS_ADJ O O O O O O O O B-BANK I-BANK O...,0
2,2,O O O O O O O O O B-PRODUCT I-PRODUCT B-COMMEN...,2
3,3,O O O B-PRODUCT I-PRODUCT O O O O B-PRODUCT O ...,2
4,4,O O O O O O B-BANK I-BANK O O O O,0
...,...,...,...
2878,2878,O O O O O O O O O O O O B-PRODUCT I-PRODUCT O O,2
2879,2879,O O O B-COMMENTS_ADJ I-COMMENTS_ADJ O O O O B-...,2
2880,2880,O O O O O O O B-PRODUCT I-PRODUCT O O O O O O ...,2
2881,2881,O B-PRODUCT I-PRODUCT I-PRODUCT B-COMMENTS_N I...,0


In [12]:
test_result.to_csv('./baseline2.csv', index=False)

In [15]:
df = pd.read_csv('ner_work/train_data_public.csv')
df['class'].value_counts()
# 对于样本不均衡， kappa 敏感性指标
# label=2 造成偏袒 => Loss weight降低
# weight
# loss
# BERT Classification 纯情感分类score = 0.41 (没对loss优化)
# BERT Classification 纯情感分类score > 0.41 (loss优化)


2    6445
0     795
1     288
Name: class, dtype: int64